<a href="https://colab.research.google.com/github/bijili-parasa/DL_PROJECTS/blob/main/project_1%2C_tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
!wget https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip

--2023-07-10 07:28:56--  https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/5721wcs2guuykzl/stacksample.zip [following]
--2023-07-10 07:28:56--  https://www.dropbox.com/s/raw/5721wcs2guuykzl/stacksample.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc0ce73bf772aaeb82700e92e85.dl.dropboxusercontent.com/cd/0/inline/B_kz8BOtxPT2G9DKET4-w1ho5FdCN-RmVMtUO4Y79jr1QZdIqqhIzTKslsBbGdtnSG90zYSwZzCK3OM5_ZIznEpUYD6Ga6tBnmaFa_XUekHA7bwK5ItAyW8gCEYK0eZO5wKp8gSHqL1j8PJSyuiwNenmM1jfpVsiKUJScN0a_hTKnA/file# [following]
--2023-07-10 07:28:57--  https://ucc0ce73bf772aaeb82700e92e85.dl.dropboxusercontent.com/cd/0/inline/B_kz8BOtxPT2G9DKET4-w1ho5FdCN-RmVMtUO4Y79jr1QZdIqqhIzTKslsBbGdtnSG90zYS

In [ ]:
!unzip stacksample.zip

Archive:  stacksample.zip
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model

In [ ]:
!rm /content/stacksample.zip

In [ ]:
df_q = pd.read_csv('/content/Questions.csv',encoding = 'latin-1')

In [ ]:
df_t = pd.read_csv('/content/Tags.csv',encoding = 'latin-1')

In [ ]:
df_t['Tag'] = df_t['Tag'].astype(str)

In [ ]:
# group all tags given to same question into a single string
grouped_tags = df_t.groupby('Id')['Tag'].apply(lambda tags: ' '.join(tags))

grouped_tags.head()

Id
80                            flex actionscript-3 air
90       svn tortoisesvn branch branching-and-merging
120                               sql asp.net sitemap
180    algorithm language-agnostic colors color-space
260           c# .net scripting compiler-construction
Name: Tag, dtype: object

In [ ]:
# reset index for simplicity
grouped_tags.reset_index()

df_tags_final = pd.DataFrame({'Id': grouped_tags.index, 'Tags': grouped_tags.values})

df_tags_final.head()

,Id,Tags
0,80,flex actionscript-3 air
1,90,svn tortoisesvn branch branching-and-merging
2,120,sql asp.net sitemap
3,180,algorithm language-agnostic colors color-space
4,260,c# .net scripting compiler-construction


In [ ]:
# Drop unnecessary columns from questions
df_q.drop(columns=['OwnerUserId', 'CreationDate', 'ClosedDate'], inplace=True)

In [ ]:
df = df_q.merge(df_tags_final, on='Id')

df.head()

,Id,Score,Title,Body,Tags
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,flex actionscript-3 air
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,svn tortoisesvn branch branching-and-merging
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql asp.net sitemap
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,algorithm language-agnostic colors color-space
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,c# .net scripting compiler-construction


In [ ]:
df = df[df['Score'] > 5]

print(df.shape)

(72950, 5)


In [ ]:
df['Tags'] = df['Tags'].apply(lambda tags: tags.lower().split())

df.head()

,Id,Score,Title,Body,Tags
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]"
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch, branching-and-merging]"
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net, sitemap]"
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,"[algorithm, language-agnostic, colors, color-s..."
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net, scripting, compiler-construction]"


In [ ]:
all_tags = []

for tags in df['Tags'].values:
    for tag in tags:
        all_tags.append(tag)

print(all_tags)

['flex', 'actionscript-3', 'air', 'svn', 'tortoisesvn', 'branch', 'branching-and-merging', 'sql', 'asp.net', 'sitemap', 'algorithm', 'language-agnostic', 'colors', 'color-space', 'c#', '.net', 'scripting', 'compiler-construction', 'c++', 'oop', 'class', 'nested-class', '.net', 'web-services', 'sql-server', 'sql-server-2005', 'deployment', 'release-management', 'c#', 'visual-studio', 'versioning', 'windows', 'visual-studio', 'registry', 'installation', 'c#', 'database', 'loops', 'connection', 'c#', 'asp.net', 'c#', 'file-locking', 'unix', 'size', 'svn', 'msbuild', 'cruisecontrol.net', 'web-applications', 'dns', 'subdomain', 'account', '.net', '.net-3.5', 'msbuild', 'nant', 'sql-server', 'windows-server-2008', 'sql-server', 'sql-server-2008', 'sql-server', 'c#', '.net', 'unit-testing', 'testing', 'svn', 'version-control', 'sql', 'database', 'postgresql', 'stored-procedures', 'triggers', 'sql', 'sql-server', 'c#', 'dataset', 'datatable', 'asp-classic', 'vbscript', 'html', 'autocomplete', 

In [ ]:
import nltk

In [ ]:
tag_freq = nltk.FreqDist(list(all_tags))

In [ ]:
tag_freq.most_common(10)

[('c#', 6907),
 ('java', 6862),
 ('javascript', 5560),
 ('android', 5052),
 ('python', 4442),
 ('c++', 4369),
 ('php', 3024),
 ('jquery', 2770),
 ('.net', 2705),
 ('ios', 2685)]

In [ ]:
tag_features = list(map(lambda x: x[0], tag_freq.most_common(10)))

print(tag_features)

['c#', 'java', 'javascript', 'android', 'python', 'c++', 'php', 'jquery', '.net', 'ios']


In [ ]:
def keep_common(tags):

    filtered_tags = []

    # filter tags
    for tag in tags:
        if tag in tag_features:
            filtered_tags.append(tag)

    # return the filtered tag list
    return filtered_tags

In [ ]:
df['Tags'] = df['Tags'].apply(lambda tags: keep_common(tags))

In [ ]:
df['Tags'] = df['Tags'].apply(lambda tags: tags if len(tags) > 0 else None)

In [ ]:
df.dropna(subset=['Tags'], inplace=True)

df.shape

(39676, 5)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
from nltk.stem.snowball import SnowballStemmer

tokenizer = ToktokTokenizer()
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))


In [ ]:
def remove_html(text):
    return re.sub(r"\<[^\>]\>", "", text).lower()

In [ ]:
def remove_stopwords(text):
    words = tokenizer.tokenize(text)

    filtered = [w for w in words if not w in stop_words]
    return ' '.join(map(str, filtered))

In [ ]:
def remove_punc(text):
    tokens = tokenizer.tokenize(text)


    tokens = list(map(lambda token: re.sub(r"[^A-Za-z0-9]+", " ", token).strip(), tokens))


    tokens = list(filter(lambda token: token, tokens))

    return ' '.join(map(str, tokens))

In [ ]:
from nltk.stem.snowball import SnowballStemmer
tokenizer = ToktokTokenizer()
stemmer = SnowballStemmer('english')

In [ ]:
def stem_text(text):

    tokens = tokenizer.tokenize(text)


    tokens = list(map(lambda token: stemmer.stem(token), tokens))

    return ' '.join(map(str, tokens))

In [ ]:
df.drop(columns=['Id', 'Score'], inplace=True)

df.head()

,Title,Body,Tags
4,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net]"
5,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,[c++]
6,Homegrown consumption of web services,<p>I've been writing a few web services for a ...,[.net]
8,Automatically update version number,<p>I would like the version property of my app...,[c#]
10,How do I connect to a database and loop over a...,<p>What's the simplest way to connect and quer...,[c#]


In [ ]:
df['Title'] = df['Title'].apply(lambda x: remove_html(x))
df['Title'] = df['Title'].apply(lambda x: remove_stopwords(x))
df['Title'] = df['Title'].apply(lambda x: remove_punc(x))
df['Title'] = df['Title'].apply(lambda x: stem_text(x))

df.head()

,Title,Body,Tags
4,ad script function net applic,<p>I have a little game written in C#. It uses...,"[c#, .net]"
5,use nest class case,<p>I am working on a collection of classes use...,[c++]
6,homegrown consumpt web servic,<p>I've been writing a few web services for a ...,[.net]
8,automat updat version number,<p>I would like the version property of my app...,[c#]
10,connect databas loop recordset c,<p>What's the simplest way to connect and quer...,[c#]


In [ ]:
df['Body'] = df['Body'].apply(lambda x: remove_html(x))
df['Body'] = df['Body'].apply(lambda x: remove_stopwords(x))
df['Body'] = df['Body'].apply(lambda x: remove_punc(x))
df['Body'] = df['Body'].apply(lambda x: stem_text(x))

df.head()

,Title,Body,Tags
4,ad script function net applic,littl game written c use databas back end a hr...,"[c#, .net]"
5,use nest class case,work collect class use video playback record o...,[c++]
6,homegrown consumpt web servic,write web servic net app readi consum them see...,[.net]
8,automat updat version number,would like version properti applic increment b...,[c#]
10,connect databas loop recordset c,simplest way connect queri databas set record c p,[c#]


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack

In [ ]:
X_title = df['Title']
X_body = df['Body']
y = df['Tags']

In [ ]:
binarizer = MultiLabelBinarizer()
y_bin = binarizer.fit_transform(y)

In [ ]:
vectorizer_title = TfidfVectorizer(
    analyzer = 'word',
    strip_accents = None,
    encoding = 'utf-8',
    preprocessor=None,
    max_features=10000)

vectorizer_body = TfidfVectorizer(
    analyzer = 'word',
    strip_accents = None,
    encoding = 'utf-8',
    preprocessor=None,
    max_features=10000)


In [ ]:
X_title_vect = vectorizer_title.fit_transform(X_title)
X_body_vect = vectorizer_body.fit_transform(X_body)

X = hstack([X_title_vect, X_body_vect])

In [ ]:
X.shape

(39676, 20000)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_bin, test_size = 0.2, random_state = 0)

In [ ]:
!pip install scikit-multilearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 9.9 MB/s eta 0:00:00


In [ ]:
from sklearn.svm import LinearSVC
from skmultilearn.problem_transform import BinaryRelevance # gives better precision

svc = LinearSVC()
clf = BinaryRelevance(svc)

# fit training data
clf.fit(X_train, y_train)

BinaryRelevance(classifier=LinearSVC(), require_dense=[True, True])

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, hamming_loss, f1_score

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
# calculate Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# calculate recall
print("Recall:", recall_score(y_true=y_test, y_pred=y_pred, average='weighted'))

# calculate precision
print("Precision: ", precision_score(y_true=y_test, y_pred=y_pred, average='weighted'))

# calculate hamming loss
print("Hamming Loss (%): ", hamming_loss(y_pred, y_test)*100)

# calculate F1 score
print("F1 Score: ", f1_score(y_pred, y_test, average='weighted'))

Accuracy: 0.7057711693548387
Recall: 0.7687861271676301
Precision:  0.8787531139319097
Hamming Loss (%):  3.676915322580645
F1 Score:  0.8271651927125935
